In [1]:
import pandas as pd
import time
import random
import os
import json
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import atexit

class NewsScraperVN:
    def __init__(self, headless=True):
        self.setup_driver(headless)
        self.data_dir = "crawled_data"
        self.all_articles_file = "all_articles.csv"
        os.makedirs(self.data_dir, exist_ok=True)
        
        # Danh sách nguồn và categories cố định
        self.sources = ["vnexpress", "tuoitre"]
        self.available_categories = {
            "vnexpress": [
                "thoi-su", "kinh-doanh", "giai-tri", "the-thao", "phap-luat", 
                "giao-duc", "suc-khoe", "doi-song", "du-lich", "khoa-hoc", 
                "so-hoa", "xe", "y-kien", "tam-su", "thu-gian", "the-gioi"
            ],
            "tuoitre": [
                "thoi-su", "kinh-te", "giai-tri", "the-thao", "phap-luat", 
                "giao-duc", "suc-khoe", "van-hoa", "du-lich", "xe", 
                "nhip-song-tre", "the-gioi", "cong-nghe", "ban-doc"
            ]
        }
        
        # Tạo thư mục cho từng nguồn
        for source in self.sources:
            os.makedirs(f"{self.data_dir}/{source}", exist_ok=True)
        
        # Đăng ký hàm để đảm bảo driver sẽ được đóng khi chương trình kết thúc
        atexit.register(self.close)
        
        # Tạo file all_articles.csv nếu chưa tồn tại
        if not os.path.exists(self.all_articles_file):
            pd.DataFrame(columns=[
                'url', 'source', 'crawl_time', 'category', 'title', 
                'description', 'publish_time', 'content', 'author'
            ]).to_csv(self.all_articles_file, index=False, encoding="utf-8-sig")
            print(f"Tạo file {self.all_articles_file} mới")
        else:
            print(f"File {self.all_articles_file} đã tồn tại, sẽ ghi thêm vào file này")
    
    def setup_driver(self, headless=True):
        """Thiết lập trình duyệt Chrome"""
        service = Service(ChromeDriverManager().install())
        
        chrome_options = webdriver.ChromeOptions()
        if headless:
            chrome_options.add_argument('--headless=new')
        
        # Cài đặt Chrome options
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('window-size=1920x1080')
        chrome_options.add_argument('--disable-notifications')
        chrome_options.add_argument('--ignore-certificate-errors')
        chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
        
        self.driver = webdriver.Chrome(service=service, options=chrome_options)
        self.driver.implicitly_wait(10)
        self.wait = WebDriverWait(self.driver, 30)
    
    def random_sleep(self, min_time=1, max_time=3):
        """Tạm dừng ngẫu nhiên giữa các yêu cầu"""
        time.sleep(random.uniform(min_time, max_time))
    
    def get_article_urls(self, source, category, num_pages=5):
        """Thu thập URL bài viết từ danh mục"""
        article_urls = set()
        
        # Cấu hình URL và selectors
        configs = {
            "vnexpress": {
                "base_url": f"https://vnexpress.net/{category}",
                "page_format": f"https://vnexpress.net/{category}-p{{}}",
                "selector": ".item-news h3 a, .title-news a"
            },
            "tuoitre": {
                "base_url": f"https://tuoitre.vn/{category}",
                "page_format": f"https://tuoitre.vn/{category}/trang-{{}}.html",
                "selector": ".box-category-item h3 a, .box-category-item h2 a, .title-news a"
            }
        }
        
        config = configs[source]
        
        # Hàm trợ giúp thu thập URL từ một trang
        def collect_urls_from_page(page_url):
            try:
                self.driver.get(page_url)
                self.random_sleep(1, 2)
                
                # Thử lấy theo selector cụ thể
                try:
                    links = self.driver.find_elements(By.CSS_SELECTOR, config["selector"])
                except:
                    # Fallback: Tìm tất cả thẻ a
                    links = self.driver.find_elements(By.TAG_NAME, "a")
                
                # Thu thập URLs
                new_urls = set()
                for link in links:
                    href = link.get_attribute("href")
                    if self.is_valid_article_url(href, source):
                        new_urls.add(href)
                
                return new_urls
            except Exception as e:
                print(f"Lỗi khi thu thập URLs từ {page_url}: {str(e)}")
                return set()
        
        # Thu thập từ trang đầu tiên
        urls_from_first_page = collect_urls_from_page(config["base_url"])
        article_urls.update(urls_from_first_page)
        print(f"Tìm thấy {len(urls_from_first_page)} liên kết trên trang 1")
        
        # Thu thập từ các trang tiếp theo
        for page in range(2, num_pages + 1):
            page_url = config["page_format"].format(page)
            urls_from_page = collect_urls_from_page(page_url)
            article_urls.update(urls_from_page)
            print(f"Tìm thấy {len(urls_from_page)} liên kết trên trang {page}, tổng cộng: {len(article_urls)}")
        
        return list(article_urls)
    
    def is_valid_article_url(self, url, source):
        """Kiểm tra URL bài viết có hợp lệ không"""
        if not url:
            return False
            
        invalid_patterns = ["/video", "/infographics", "/media", "javascript:"]
        
        # Kiểm tra và loại bỏ URLs không hợp lệ
        if any(pattern in url for pattern in invalid_patterns):
            return False
            
        # Kiểm tra định dạng URL theo nguồn
        if source == "vnexpress":
            return "https://vnexpress.net/" in url and ".html" in url
        elif source == "tuoitre":
            return "https://tuoitre.vn/" in url and ".htm" in url
            
        return False
    
    def scrape_article(self, url, source):
        """Thu thập nội dung bài viết với cấu trúc tối ưu hóa"""
        try:
            self.driver.get(url)
            self.random_sleep(1, 2)
            
            # Chờ trang tải xong
            self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1, .title-detail, .article-title")))
            
            # Cấu hình selectors cho từng nguồn
            selectors = {
                "vnexpress": {
                    "title": "h1.title-detail, h1.title-news",
                    "description": ".description, .lead_detail",
                    "time": ".date, .time, time.date",
                    "content": ".fck_detail .Normal, .fck_detail p, .fck_detail",
                    "author": ".author_mail, .author, .author_name"
                },
                "tuoitre": {
                    "title": "h1.article-title, h1.title-page, h1.detail-title",
                    "description": ".sapo, .summary, .article-sapo",
                    "time": ".date-time, time.op-published",
                    "content": ".detail-content, .article-content, #main-detail-body",
                    "author": ".author, .author-info, .name-author"
                }
            }
            
            # Khởi tạo thông tin bài viết
            article = {
                "url": url,
                "source": source,
                "crawl_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "category": ""
            }
            
            # Hàm trợ giúp lấy nội dung theo selector
            def get_element_text(selector_str):
                for selector in selector_str.split(','):
                    try:
                        element = self.driver.find_element(By.CSS_SELECTOR, selector.strip())
                        return element.text.strip()
                    except:
                        continue
                return ""
            
            # Thu thập dữ liệu
            article["title"] = get_element_text(selectors[source]["title"])
            article["description"] = get_element_text(selectors[source]["description"])
            article["publish_time"] = get_element_text(selectors[source]["time"])
            article["author"] = get_element_text(selectors[source]["author"])
            
            # Thu thập nội dung
            try:
                content_selector = selectors[source]["content"]
                paragraphs = []
                
                # Thử lấy nội dung từ các thẻ p
                for selector in content_selector.split(','):
                    try:
                        content_element = self.driver.find_element(By.CSS_SELECTOR, selector.strip())
                        p_elements = content_element.find_elements(By.TAG_NAME, "p")
                        if p_elements:
                            paragraphs = [p.text.strip() for p in p_elements if p.text.strip()]
                            break
                    except:
                        continue
                
                # Nếu không có thẻ p, lấy toàn bộ nội dung
                if not paragraphs:
                    content_text = get_element_text(content_selector)
                    paragraphs = [line.strip() for line in content_text.split('\n') if line.strip()]
                
                article["content"] = "\n".join(paragraphs)
            except Exception as e:
                article["content"] = ""
                print(f"Lỗi khi lấy nội dung: {url}, {str(e)}")
            
            # Xác định category từ URL
            for cat in self.available_categories[source]:
                if any(pattern in url for pattern in [f"/{cat}/", f"/{cat}.", f"/{cat}-"]):
                    article["category"] = cat
                    break
            
            # Kiểm tra tính hợp lệ của bài viết
            if not article["title"] or not article["content"]:
                print(f"Bài viết không đủ thông tin: {url}")
                return None
                
            return article
            
        except TimeoutException:
            print(f"Timeout khi tải trang: {url}")
            return None
        except Exception as e:
            print(f"Lỗi khi thu thập nội dung: {url}, {str(e)}")
            return None
    
    def append_to_all_articles(self, article):
        """Thêm bài viết vào file tổng hợp all_articles.csv ngay lập tức"""
        try:
            # Tạo DataFrame từ article và ghi vào file
            df = pd.DataFrame([article])
            df.to_csv(self.all_articles_file, mode='a', header=False, index=False, encoding="utf-8-sig")
            # Đảm bảo dữ liệu được ghi vào đĩa ngay lập tức
            return True
        except Exception as e:
            print(f"Lỗi khi ghi vào all_articles.csv: {str(e)}")
            return False
    
    def get_crawled_urls(self):
        """Lấy danh sách URLs đã crawl từ file all_articles.csv"""
        crawled_urls = set()
        if os.path.exists(self.all_articles_file) and os.path.getsize(self.all_articles_file) > 0:
            try:
                existing_df = pd.read_csv(self.all_articles_file, encoding="utf-8-sig")
                if 'url' in existing_df.columns:
                    crawled_urls = set(existing_df['url'].tolist())
                    print(f"Đã tìm thấy {len(crawled_urls)} URLs đã crawl trước đó")
            except Exception as e:
                print(f"Lỗi khi đọc file all_articles.csv: {str(e)}")
        return crawled_urls
    
    def scrape_category(self, source, category, num_pages=5, max_articles=50):
        """Thu thập bài viết từ một danh mục và lưu ngay lập tức vào all_articles.csv"""
        if category not in self.available_categories[source]:
            print(f"Danh mục '{category}' không hợp lệ cho {source}")
            return []

        print(f"\n=== Crawl {source} - {category} ===")
        
        # Tạo thư mục lưu trữ
        category_dir = f"{self.data_dir}/{source}/{category}"
        os.makedirs(category_dir, exist_ok=True)
        
        # Lấy URLs đã crawl
        crawled_urls = self.get_crawled_urls()
        
        # Thu thập URLs mới
        article_urls = self.get_article_urls(source, category, num_pages)
        
        # Lọc URLs chưa crawl
        new_urls = [url for url in article_urls if url not in crawled_urls]
        print(f"Tìm thấy {len(new_urls)} URLs mới chưa crawl")
        
        # Giới hạn số lượng bài viết
        if max_articles and len(new_urls) > max_articles:
            new_urls = new_urls[:max_articles]
            print(f"Giới hạn crawl tối đa {max_articles} URLs")

        articles = []
        skipped = 0
        
        # Progress bar
        with tqdm(total=len(new_urls), desc=f"Crawl {source}/{category}") as pbar:
            for url in new_urls:
                try:
                    # Crawl bài viết
                    article = self.scrape_article(url, source)
                    
                    if article:
                        # Kiểm tra category
                        if article["category"] != category:
                            if article["category"]:
                                skipped += 1
                                pbar.update(1)
                                continue
                            article["category"] = category
                        
                        # Lưu bài viết dạng JSON
                        article_id = url.split("/")[-1].split(".")[0]
                        json_path = f"{category_dir}/{article_id}.json"
                        with open(json_path, "w", encoding="utf-8") as f:
                            json.dump(article, f, ensure_ascii=False, indent=2)
                        
                        # GHI NGAY LẬP TỨC: Thêm vào file all_articles.csv
                        self.append_to_all_articles(article)
                        articles.append(article)
                        
                        # In thông báo mỗi khi crawl thành công
                        pbar.set_postfix_str(f"Đã ghi: {article['title'][:30]}...")
                    
                    pbar.update(1)
                except Exception as e:
                    print(f"\nLỗi khi xử lý {url}: {str(e)}")
                    pbar.update(1)
                    continue
                
                # Tạm dừng ngẫu nhiên để tránh bị chặn
                self.random_sleep(1, 2)

        print(f"Đã crawl {len(articles)} bài viết từ {source}/{category}")
        if skipped > 0:
            print(f"Đã bỏ qua {skipped} bài viết không thuộc danh mục {category}")

        return articles

    def scrape_multiple_categories(self, config):
        """Thu thập từ nhiều danh mục"""
        total_articles = 0
        start_time = datetime.now()
        
        print(f"Bắt đầu crawl vào: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        
        for source, categories in config.items():
            for cat_info in categories:
                category = cat_info["category"]
                num_pages = cat_info.get("pages", 5)
                max_articles = cat_info.get("max_articles", 50)
                
                # Thu thập và ghi ngay lập tức vào all_articles.csv
                articles = self.scrape_category(source, category, num_pages, max_articles)
                total_articles += len(articles)
                
                print(f"Hoàn tất crawl {source}/{category}: {len(articles)} bài viết")
                print(f"Tổng cộng đã crawl: {total_articles} bài viết")
                print("-" * 50)
        
        end_time = datetime.now()
        duration = end_time - start_time
        print(f"\nHoàn tất crawl tổng cộng: {total_articles} bài viết")
        print(f"Thời gian thực hiện: {duration}")
        return total_articles

    def close(self):
        """Đóng trình duyệt"""
        if hasattr(self, "driver"):
            try:
                self.driver.quit()
                print("Đã đóng trình duyệt")
            except:
                pass

# Sử dụng
if __name__ == "__main__":
    config = {
        # "vnexpress": [
        #     {"category": "thoi-su", "pages": 1, "max_articles": 50},
            # {"category": "suc-khoe", "pages": 5, "max_articles": 100},
        #     {"category": "kinh-doanh", "pages": 1, "max_articles": 50},
            # {"category": "phap-luat", "pages": 6, "max_articles": 200},
            # {"category": "giao-duc", "pages": 10, "max_articles": 200},
            # {"category": "khoa-hoc", "pages": 10, "max_articles": 300},
        # ],  
        "tuoitre": [
            # {"category": "thoi-su", "pages": 1, "max_articles": 50},
            # {"category": "suc-khoe", "pages": 1, "max_articles": 50},
            {"category": "kinh-te", "pages": 10, "max_articles": 500},
            {"category": "phap-luat", "pages": 10, "max_articles":110},
            {"category": "giao-duc", "pages":10, "max_articles": 100},
            {"category": "cong-nghe", "pages": 10, "max_articles": 200},
        ]
    }

    try:
        scraper = NewsScraperVN(headless=True)
        scraper.scrape_multiple_categories(config)
    except KeyboardInterrupt:
        print("\nNgắt bởi người dùng. Dữ liệu đã được lưu liên tục vào all_articles.csv")
    except Exception as e:
        print(f"Lỗi tổng quát: {str(e)}")
    finally:
        if 'scraper' in locals():
            scraper.close()

File all_articles.csv đã tồn tại, sẽ ghi thêm vào file này
Bắt đầu crawl vào: 2025-03-14 14:24:32

=== Crawl tuoitre - kinh-te ===
Đã tìm thấy 1491 URLs đã crawl trước đó
Tìm thấy 110 liên kết trên trang 1
Tìm thấy 10 liên kết trên trang 2, tổng cộng: 120
Tìm thấy 10 liên kết trên trang 3, tổng cộng: 130
Tìm thấy 10 liên kết trên trang 4, tổng cộng: 140
Tìm thấy 10 liên kết trên trang 5, tổng cộng: 150
Tìm thấy 10 liên kết trên trang 6, tổng cộng: 160
Tìm thấy 10 liên kết trên trang 7, tổng cộng: 170
Tìm thấy 10 liên kết trên trang 8, tổng cộng: 180
Tìm thấy 10 liên kết trên trang 9, tổng cộng: 190
Tìm thấy 10 liên kết trên trang 10, tổng cộng: 200
Tìm thấy 184 URLs mới chưa crawl


Crawl tuoitre/kinh-te:   3%|▎         | 5/184 [07:16<5:12:05, 104.61s/it, Đã ghi: VinUni và NTU - Singapore ký k...]

Bài viết không đủ thông tin: https://tuoitre.vn/kiem-tien-bang-nghe-doc-la-hong-giong-ai-e1918.htm


Crawl tuoitre/kinh-te:   4%|▍         | 8/184 [12:24<5:30:51, 112.79s/it, Đã ghi: TP.HCM, Nam Bộ nắng nóng như ‘...]

Bài viết không đủ thông tin: https://tuoitre.vn/tam-dai-dong-nganh-o-trung-quoc-2024101015305498.htm


...

Bài viết không đủ thông tin: https://tuoitre.vn/de-den-ngay-dac-vu-ai-quan-xuyen-moi-thu-20241211084845322.htm


Crawl tuoitre/cong-nghe:  96%|█████████▌| 49/51 [1:02:46<03:47, 113.97s/it, Đã ghi: Sau làn sóng tăng giá ảo, giới...]

Bài viết không đủ thông tin: https://tuoitre.vn/cong-nghe-thoi-huong-trump-20250117094018516.htm


Crawl tuoitre/cong-nghe: 100%|██████████| 51/51 [1:04:57<00:00, 76.42s/it, Đã ghi: Tổng Bí thư Tô Lâm: 'Lãnh đạo ...] 


Đã crawl 43 bài viết từ tuoitre/cong-nghe
Đã bỏ qua 1 bài viết không thuộc danh mục cong-nghe
Hoàn tất crawl tuoitre/cong-nghe: 43 bài viết
Tổng cộng đã crawl: 241 bài viết
--------------------------------------------------

Hoàn tất crawl tổng cộng: 241 bài viết
Thời gian thực hiện: 6:47:57.367138
Đã đóng trình duyệt
